# 27、使用K-Means对图像进行划分

聚类的一个常用的场景就是对图像进行划分。

图像划分就是利用图像自身的信息，比如颜色、纹理、形状等特征进行划分，将图像分割成为不同的区域，划分出来的每个区域就是相当于是对图像中的像素进行了聚类。单个区域内部的像素之间的相似度很大，不同的区域间的像素差异大。这个特性正好符合聚类的特性，所以你可以把图像分割成为是将图像中的信息进行聚类。当然聚类只是分割图像的一种方式，除了聚类，我们还可以基于图像颜色的阈值进行分割，或者基于图像边缘的信息进行分割等。

## 将微信开屏的封面进行分割

我们可以使用sklearn中的K-Means算法岁微信页面进行分割，其开屏的画面如下：

![](微信开屏.png)

聚类的流程和分类差不多，设定流程如下：
![](图像聚类流程.png)

在准备阶段里面，我们需要对数据进行加载。因为处理的是图像信息，我们除了需要获取到图像数据以外，还需要获取图像的尺寸和通道数，然后基于图像的每个通道的数值进行数据的规范化。这里我们需要定义个函数load_data,来帮助我们进行图像的加载和数据的规范化。代码如下所示：

In [ ]:
# 加载图像，并对数据进行规范化
def load_data(filePath):
    f = open(filePath,'rb')
    data = []
    # 得到图像的像素值
    img = image.open(f)
    # 得到图像的尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 得到点(x,y)的三个通道值
            c1, c2, c3 = img.getpixel((x,y))
            data.append([c1,c2,c3])
    f.close()
    # 采用Min-Max规范化
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    return np.mat(data), width, height

因为jpg格式的图像是三个通道(R,G,B),也就是一个像素点具有3个特征值。这里我们使用c1、c2、c3来获取平面坐标点(x,y)的三个特征值，特征值是在0-255之间。

为了加快聚类的收敛，我们采用的是Min-max规范化对数据进行规范化。我们定义的load_data函数返回的结果包含了针对(R,G,B)三个通道规范化的数据，以及图像的尺寸信息。在定义好load_data函数之后，我们直接调用就可以得到相关的信息，代码如下：

In [ ]:
# 加载图像，得到规范化的结果 img，以及图像的尺寸
img, width, height = load_data('./weixin.png')

假设我们想要对图像分割成为两部分，在聚类阶段，我们可以将聚类设置成为2，这样图像就自动聚成2类，代码如下：

In [ ]:
# 用K-Means对图像进行2聚类
kmeans = KMeans(n_clusters=2)
kmeans.fit(img)
label = kmeans.predict(img)
# 将图像聚类结果，转化成为图像的尺寸的矩阵
label = label.reshape([width, height])
# 创建个新的图像 pic_mark,用来保存图像的聚类的结果，并设置不同的灰度值
pic_mark = image.new("L", (width, height))
for x in range(width):
    for y in range(height):
        # 根据类别设置图像的灰度，类别 0灰度值为255， 类别1灰度值为127
        pic_mark.putpixel((x,y), int(256/(label[x][y]+1))-1)
pic_mark.save("weixin_mark.jpg", "JPEG")
        

一些参数的解析，作用和设置方法。

使用了fit和predict两个函数来做数据的训练拟合和预测，因为传入的参数是一样的，所以我们可以同时进行fit和predict操作，这样我们就可以直接使用fit_predict(data)得到聚类的结果。得到聚类的结果label之后，实际上是一个一维向量，我们需要把它转化成为图像尺寸的矩阵。label的聚类结果是从0开始统计的，当聚类数为2的时候，聚类的标识label=0或者1。

如果你想对图像聚类的结果进行可视化，直接看0和1是肯不出来的，还需要将0和1转化为灰度值。灰度值一般是在0-255的范围内，我们可以将label=0设定位灰度值255，label=1，设定为灰度值为127.具体的方法使用int(256/(label[x][y]+1))-1。可视化的时候，主要是通过设置图像的灰度值进行显示。所以我们把聚类label=0的像素点都统一设置为灰度值为255，把聚类label=1的像素点都统一设置为灰度值为127.原来的像素的灰度值是在0-255之间，现在就只有2种颜色（也就是灰度值为255，和灰度为127）。

有了这些灰度信息，我们就可以用image.new创建一个新的图像，用putpixel函数对新图像的点进行灰度值的设置，最后用save函数保存聚类的灰度图像。这样就可以看到聚类的可视化结果了：

In [41]:
# -*- coding:utf-8 -*-
# 使用K-Means对图像进行聚类，显示分割标识的可视化
import numpy as np
# 使用pillow包里面的图像模块
import PIL.Image as image
from sklearn.cluster import KMeans
from sklearn import preprocessing

# 加载图像，并对数据进行规范化
def load_data(filePath):
    f = open(filePath,'rb')
    data = []
    # 得到图像的像素值
    img = image.open(f)
    # 得到图像的尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 得到点(x,y)的三个通道值
            c1, c2, c3 = img.getpixel((x,y))
            data.append([c1,c2,c3])
    f.close()
    # 采用Min-Max规范化
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    return np.mat(data), width, height

# 加载图像，得到规范化的结果 img，以及图像的尺寸
img, width, height = load_data('./weixin.jpg')

# 用K-Means对图像进行2聚类
kmeans = KMeans(n_clusters=2)
kmeans.fit(img)
label = kmeans.predict(img)
# 将图像聚类结果，转化成为图像的尺寸的矩阵
label = label.reshape([width, height])
# 创建个新的图像 pic_mark,用来保存图像的聚类的结果，并设置不同的灰度值
pic_mark = image.new("L", (width, height))
for x in range(width):
    for y in range(height):
        # 根据类别设置图像的灰度，类别 0灰度值为255， 类别1灰度值为127
        pic_mark.putpixel((x,y), int(256/(label[x][y]+1))-1)
pic_mark.save("weixin_mark.jpg", "JPEG")


如果我们想要将图像分割成为16个部分，应该如何设置不同的颜色值？这里使用到的是skimage工具包，它是图像处理工具包。需要另外安装：

In [35]:
sudo pip3 install -U scikit-image

这段代码可以将聚类表示转换成为不同颜色的矩阵：

In [ ]:
from skimage import color 
import numpy as np
# 将聚类标识矩阵转化成为不同颜色的矩阵
label_color = (color.label2rgb(label)*255).astype(np.uint8)
# 需要将第一维和第二维颠倒过来
label_color = label_color.transpose(1,0,2)
images = image.fromarray(label_color)
images.save('weixin_mark_color.jpg')

代码中，使用了skimage中的label2rgb函数来将label分类标识转化成为颜色数值，因为我们的颜色值的范围是[0,255],所以还需要乘以255进行转化，最后再转化为np.uint8类型。uint8类型代表无符号的整数，范围在0-255之间。

得到颜色矩阵之后，我们可以把它输出来，这时候你会发现输出的图像是颠倒的，原因可能是图像源拍摄的时候本身是倒置的。我们需要设置三维矩阵的装置，让第一维和第二维颠倒过来，也就是使用transpose(1,0,2),将原来的(0,1,2)顺序转化成为(1,0,2)顺序，即第一维和第二维互换。

最后我们使用fromarray函数，它可以通过矩阵来生成图片，并使用save进行保存。

最后得到的分类标识的颜色化图像如下所示：

In [45]:
# -*- coding:utf-8 -*-
# 使用K-Means对图像进行聚类，显示分割标识的可视化
import numpy as np
# 使用pillow包里面的图像模块
import PIL.Image as image
from sklearn.cluster import KMeans
from sklearn import preprocessing
from skimage import color 

# 加载图像，并对数据进行规范化
def load_data(filePath):
    f = open(filePath,'rb')
    data = []
    # 得到图像的像素值
    img = image.open(f)
    # 得到图像的尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 得到点(x,y)的三个通道值
            c1, c2, c3 = img.getpixel((x,y))
            data.append([c1,c2,c3])
    f.close()
    # 采用Min-Max规范化
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    return np.mat(data), width, height

# 加载图像，得到规范化的结果 img，以及图像的尺寸
img, width, height = load_data('./weixin.jpg')

# 用K-Means对图像进行2聚类
kmeans = KMeans(n_clusters=16)
kmeans.fit(img)
label = kmeans.predict(img)
# 将图像聚类结果，转化成为图像的尺寸的矩阵
label = label.reshape([width, height])
# 创建个新的图像 pic_mark,用来保存图像的聚类的结果，并设置不同的灰度值
pic_mark = image.new("L", (width, height))
for x in range(width):
    for y in range(height):
        # 根据类别设置图像的灰度，类别 0灰度值为255， 类别1灰度值为127
        pic_mark.putpixel((x,y), int(256/(label[x][y]+1))-1)
pic_mark.save("weixin_mark.jpg", "JPEG")


# 将聚类标识矩阵转化成为不同颜色的矩阵
label_color = (color.label2rgb(label)*255).astype(np.uint8)
# 需要将第一维和第二维颠倒过来
label_color = label_color.transpose(1,0,2)
images = image.fromarray(label_color)
images.save('weixin_mark_color.jpg')


上面生成的图片的样式如下所示，一张是只分成16个簇的双色灰度照片，另外一张是分成16个簇的经过skimage处理过的图片
![](weixin_mark.jpg)  ![](weixin_mark_color.jpg)


刚才我们做的是聚类的可视化，如果我们想要看到的是对应的原图，可以将每个簇（即每一个类别）的点的RGB值设置为该簇的质心点的RGB的值，也就是**该簇内的点的特征均为质心点的特征**：

就是，将原图像进行聚类，聚类完成之后，使用该类的质心点的RGB的值设置为该簇的每一个点，然后将这样的反转为新的图片，就能够清除的看出原图的聚类情况了。如下图所示：

例如下面的代码，代码中，我们可以把范围为0-255的数值映射到1-256数值之间，方法是对每个数值进行加1，可以运行：

In [47]:
#-*- coding:utf-8 -*-
# 使用K-means对图像进行聚类，并显示聚类压缩后的图像
import numpy as np
import PIL.Image as image
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.image as mpimg
# 加载图像，并对数据进行规范化
def load_data(filePath):
    # 1、读文件
    f = open(filePath,'rb')
    data = []
    # 2、得到图像的像素值
    img = image.open(f)
    # 3、得到图像的尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 4、得到点的（x,y）的三个通道值
            c1,c2,c3 = img.getpixel((x,y))
            data.append([(c1+1)/256.0, (c2+1)/256.0, (c3+1)/256.0])
    f.close()
    return np.mat(data), width, height

# 1、加载图像，得到规范化的结果 imgData，以及图像的尺寸
img, width, height = load_data('./weixin.jpg')
# 2、使用K-Means对图像进行16聚类
kmeans = KMeans(n_clusters=16)
label = kmeans.fit_predict(img)
# 3、将图像聚类结果转化成为图像尺寸的矩阵
label = label.reshape([width, height])
# 4、创建一个新的图像img，用来保存图像的聚类压缩后的结果
img=image.new('RGB', (width, height))
for x in range(width):
    for y in range(height):
        c1 = kmeans.cluster_centers_[label[x,y],0]
        c2 = kmeans.cluster_centers_[label[x,y],1]
        c3 = kmeans.cluster_centers_[label[x,y],2]
        img.putpixel((x,y), (int(c1*256)-1, int(c2*256)-1, int(c3*256)-1))
img.save('weixin_new.jpg')

由上面的代码可以看出，没有用到sklearn自带的MinMaxScaler，而是自己写了Min-Max规范化的公式。这样做的原因是我们知道RGB每个通道的数值之间在[0,255]之间，所以我们可以用每个通道的数值+1/256,这样数值就会在[0,1]之间。

对图像做了Min-Max空间变换之后，还可以对其进行反变换，还原出对应原图的通道值。

对于点(x,y),我们找到他们所在的簇label[x,y],然后得到这个簇的质心特征，用c1，c2，c3表示：

In [ ]:
c1 = kmeans.cluster_centers_[label[x,y],0]
c2 = kmeans.cluster_centers_[label[x,y],1]
c3 = kmeans.cluster_centers_[label[x,y],2]

因为c1，c2，c3对应的是数据规范化的数值，因此我们还需要进行反变换，即：

In [ ]:
c1=int(c1*256)-1
c2=int(c2*256)-1
c3=int(c3*256)-1

然后使用img.putpixel设置点（x,y）反变换之后得到的特征值。随后使用img.save保存图像。下面是原图和转换之后的图片：
![](weixin.jpg)
![](weixin_new.jpg)

## 总结

1、使用K-Means对图像进行分割，不难发现其有个缺陷：聚类个数K值需要实现指定。如果你不知道该聚成几类，那么最好会给K值多设置几个点，然后选择聚类结果最好的那个值

2、在整个图像的分割过程中，我们发现使用K-means计算的过程在sklearn中就是几行代码，大部分的工作还是在预处理和后处理的基础上。预处理是将图像进行加载，数据规范化。后处理是对决的结果进行反变换。

3、如果涉及后处理，我们也可以自己来设定数据规范化的函数，这样的反变换的函数比较容易编写。

4、另外使用Python对图像进行读写，具体代码如下，上面的代码中也有对应的实现代码：

In [ ]:
import PIL.Image as image
# 得到图像的像素值
img = image.open(f)
# 得到图像的尺寸
width, height = img.size

这里会使用到PIL这个工具包，它的英文全称叫做Python Imaging Library，就是Python的图像处理标准库。同时也使用到了skimage工具包（scikit-image），它也是图像的处理工具包。它和Matlib处理图像相媲美，集成很多的图像处理函数，其中对不同的分类标识显示不同的颜色。在Python中图像的处理工具包，使用的就是skimage工具包。

5、工具的使用和注意点
- K-Means的聚类工具，数据规范化工具，图像处理工具，应用工具进行对图像进行分割
- 不同的尺寸图像，K-Means的运行时间不同。如果尺寸较大，就可以先压缩，长度在200像素内运行的比较快，超过1000像素，速度很慢，例如本例代码

6、KMeans实战
- 聚类工具
    - 创建：kmeans=KMeans(n_cluster=16)
    - 训练：kmeans.fit(data)
    - 预测：kmeans.predict(data)
    - 训练&预测：kmeans.fit_predict(data)
- 数据规范化工具
    - Min-Max规范化：preprocessing.MinMaxScaler()
    - 数据规范化：fit_transform(data)
    - 数据反变换：在Min-Max规范化的时候，自己定义Min-Max的Max值，方便求解反变换的函数。
- 图像处理工具
    - 工具包：PIL.Image,skimage
    - 获取图像文件的内容（像素值）:Image.open(f)
    - 创建新的像素：image=Image.new("RGB", (width, height))
    - 将矩阵转化成为图像：Image.fromarray(label_color)
    - 保存图像：image.save(filename)
    - 将分类标识矩阵转化成为不同颜色的矩阵：(color.label2rgb(label)*255).astype(np.uint8)

![](KMeans实战.png)

## 小练习

In [49]:
# -*- coding:utf-8 -*-
# 使用K-Means对图像进行聚类，显示分割标识的可视化
import numpy as np
# 使用pillow包里面的图像模块
import PIL.Image as image
from sklearn.cluster import KMeans
from sklearn import preprocessing
from skimage import color 

# 加载图像，并对数据进行规范化
def load_data(filePath):
    f = open(filePath,'rb')
    data = []
    # 得到图像的像素值
    img = image.open(f)
    # 得到图像的尺寸
    width, height = img.size
    for x in range(width):
        for y in range(height):
            # 得到点(x,y)的三个通道值
            c1, c2, c3 = img.getpixel((x,y))
            data.append([c1,c2,c3])
    f.close()
    # 采用Min-Max规范化
    mm = preprocessing.MinMaxScaler()
    data = mm.fit_transform(data)
    return np.mat(data), width, height

# 加载图像，得到规范化的结果 img，以及图像的尺寸
img, width, height = load_data('./baby.jpg')

# 用K-Means对图像进行2聚类
kmeans = KMeans(n_clusters=16)
kmeans.fit(img)
label = kmeans.predict(img)
# 将图像聚类结果，转化成为图像的尺寸的矩阵
label = label.reshape([width, height])
# 创建个新的图像 pic_mark,用来保存图像的聚类的结果，并设置不同的灰度值
pic_mark = image.new("L", (width, height))
for x in range(width):
    for y in range(height):
        # 根据类别设置图像的灰度，类别 0灰度值为255， 类别1灰度值为127
        pic_mark.putpixel((x,y), int(256/(label[x][y]+1))-1)
pic_mark.save("baby_mark.jpg", "JPEG")


# 将聚类标识矩阵转化成为不同颜色的矩阵
label_color = (color.label2rgb(label)*255).astype(np.uint8)
# 需要将第一维和第二维颠倒过来
label_color = label_color.transpose(1,0,2)
images = image.fromarray(label_color)
images.save('baby_mark_color.jpg')
